In [1]:
!pip install qdrant-client langchain_ollama langchain langchain_community

In [2]:
!pip install huggingface_hub streamlit

In [3]:
from qdrant_client import QdrantClient
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_ollama import ChatOllama
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
import streamlit as st

In [4]:


qdrant_client = QdrantClient(
    url="",
    api_key="",
)

print(qdrant_client.get_collections())

collections=[]


In [5]:
from re import VERBOSE
class ChatbotManager :
  def __init__(self,model_name,device,encode_kwargs,llm_model,llm_temperature,qdrant_url,collection_name):
    self.model_name = model_name
    self.device = device
    self.encode_kwargs = encode_kwargs
    self.llm_model = llm_model
    self.llm_temperature = llm_temperature
    self.qdrant_url = qdrant_url
    self.collection_name = collection_name

    self.embeddings = HuggingFaceEmbeddings(
        model_name=self.model_name,
        model_kwargs={'device': self.device},
        encode_kwargs=self.encode_kwargs

    )

    self.llm = ChatOllama(
        model=self.llm_model,
        temperature=self.llm_temperature
    )

    self.prompt_template = """
    se the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer. Answer must be detailed and well explained.
Helpful answer:
    """

    self.client = QdrantClient(
        url = self.qdrant_url,prefer_grpc=False
    )

    self.db = Qdrant(
        client = self.client,
        embeddings = self.embeddings,
        collection_name = self.collection_name
    )

    self.prompt = PromptTemplate(
        template=self.prompt_template, input_variables=["context", "question"]
    )


    self.retriever = self.db.as_retriever(search_kwargs = {"k": 3})

    self.chain_type_kwargs = {"prompt": self.prompt}

    self.qa = RetrievalQA.from_chain_type(
        llm=self.llm,
        chain_type="stuff",
        retriever=self.retriever,
        return_source_documents= False,
        chain_type_kwargs=self.chain_type_kwargs,
        verbose = False
    )


  def get_response(self,question):
    return self.qa.run(question)


# VectorsEmbeddding

In [6]:
!pip install langchain-text-splitters

In [7]:
import base64
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Qdrant
from langchain_community.embeddings import HuggingFaceEmbeddings
from qdrant_client import QdrantClient

In [8]:
class EmbeddingsManager:
  def __init__(self,model_name,device,encode_kwargs,qdrant_url,collection_name):
    self.model_name = model_name
    self.device = device
    self.encode_kwargs = encode_kwargs
    self.qdrant_url = qdrant_url
    self.collection_name = collection_name

    self.embeddings = HuggingFaceEmbeddings(
        model_name=self.model_name,
        model_kwargs={'device': self.device},
        encode_kwargs=self.encode_kwargs

    )

  def create_embeddings(self,pdf_path):
    loader = UnstructuredPDFLoader(pdf_path)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    qdrant = Qdrant.from_documents(splits,self.embeddings,url = self.qdrant_url,prefer_frpc=False,collection_name = self.collection_name)

    return "Vector DB successfully created and stored"

## new.py

In [9]:
!pip install PyMuPDF pdfminer

In [10]:
!pip install unstructured

In [11]:
!pip install sentence-transformers

In [12]:
!pip install "pdfminer.six==20221105"
!pip install unstructured
!pip install sentence-transformers

  Using cached pdfminer.six-20221105-py3-none-any.whl.metadata (4.0 kB)
Using cached pdfminer.six-20221105-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20231228
    Uninstalling pdfminer.six-20231228:
      Successfully uninstalled pdfminer.six-20231228
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pdfplumber 0.11.4 requires pdfminer.six==20231228, but you have pdfminer-six 20221105 which is incompatible.


In [13]:
!pip install unstructured[pdf]

  Using cached pdfminer.six-20231228-py3-none-any.whl.metadata (4.2 kB)
Using cached pdfminer.six-20231228-py3-none-any.whl (5.6 MB)
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20221105
    Uninstalling pdfminer.six-20221105:
      Successfully uninstalled pdfminer.six-20221105


In [18]:
!apt-get install poppler-utils
!apt-get install tesseract-ocr
!apt-get install libtesseract-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.

In [14]:
from IPython.display import IFrame
import fitz  # PyMuPDF
import base64


from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [19]:

def displayPDF(file_path):
    """Displays a PDF file within a Jupyter Notebook using IFrame."""
    # Open the PDF using PyMuPDF
    doc = fitz.open(file_path)

    # Convert the PDF to a byte string
    pdf_bytes = doc.tobytes()

    # Embed the PDF in an IFrame
    return IFrame(src=f"data:application/pdf;base64,{base64.b64encode(pdf_bytes).decode()}", width="100%", height="600")

def main():
  file = open("/content/temp.pdf")
  displayPDF(file)
  embeddingmanager = EmbeddingsManager(
      model_name="BAAI/bge-small-en",
        device="cpu",
        encode_kwargs={"normalize_embeddings": True},
        qdrant_url="http://localhost:6333",
        collection_name="vector_db"
  )
  embeddingresult = embeddingmanager.create_embeddings("/content/temp.pdf")
  print(embeddingresult)

  #chatbot stimulator

  chatbot_manager = ChatbotManager(
      model_name = "BAAT/bge-small-en",
      device ="cpu",
      encode_kwargs={"normalize_embeddings": True},
        llm_model="llama3.2:3b",
        llm_temperature=0.7,
        qdrant_url="http://localhost:6333",
        collection_name="vector_db"

  )
  user_input = input("Type your message here: ")
  answer = chatbot_manager.get_response(user_input)
  print(answer)

if __name__ == "__main__":
    main()


OSError: No such file or directory: '/root/nltk_data/tokenizers/punkt/PY3_tab'